## Try this Notebook in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truefoundry/mlfoundry-examples/blob/main/examples/sklearn/iris_classification.ipynb)

## Install dependencies

In [35]:
! pip install --quiet "numpy>=1.0.0,<2.0.0" "pandas>=1.0.0,<2.0.0" "matplotlib>=3.5.2,<3.6.0" scikit-learn shap==0.40.0
! pip install -U "mlfoundry>=0.4.6,<0.5.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.3/127.3 kB 3.4 MB/s eta 0:00:00


  Attempting uninstall: mlfoundry
    Found existing installation: mlfoundry 0.4.5
    Uninstalling mlfoundry-0.4.5:
      Successfully uninstalled mlfoundry-0.4.5


## Initialize MLFoundry Client

In [36]:
import os
import getpass
import urllib.parse
import mlfoundry as mlf

In [38]:
client = mlf.get_client()

---

## Iris plants detection as a Classification problem

In [39]:
import urllib.parse

import shap
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

import mlfoundry as mlf

### Loading data and preprocessing

In [40]:
data = datasets.load_iris()
print(data.keys())

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])


In [41]:
print(data.DESCR) 

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [42]:
# Read the DataFrame, first using the feature data
df = pd.DataFrame(data.data, columns=data.feature_names)
# Add a target column, and fill it with the target data
df['target'] = data.target
# Show the first five rows
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### Split Dataset into Training and Validation

In [43]:
# Store the feature data
X = pd.DataFrame(data.data, columns=data.feature_names)
# store the target data
y = data.target

In [44]:
# split the data using scikit-learn's train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, stratify=y, random_state=42)
print('Train samples:', len(X_train))
print('Test samples:', len(X_test))

Train samples: 120
Test samples: 30


### Start MLFoundry Run

In [45]:
run = client.create_run(project_name='iris-clf-demo', run_name='svm-model')
print('RUN ID:', run.run_id)

Link to the dashboard for the run: https://app.truefoundry.com/mlfoundry/331/e6d5c80fab17497eb3366ac62f985fca/
[mlfoundry] 2022-10-03T18:34:47+0530 INFO Run 'truefoundry/Nikhil/iris-clf-demo/svm-model-2' has started.


Exception ignored in: <function MlFoundryRun.__del__ at 0x7fd27930b5e0>
Traceback (most recent call last):
  File "/Users/nikp18/opt/anaconda3/envs/p38/lib/python3.8/site-packages/mlfoundry/mlfoundry_run.py", line 232, in __del__
    self._terminate_run_if_running(RunStatus.FINISHED)
  File "/Users/nikp18/opt/anaconda3/envs/p38/lib/python3.8/site-packages/mlfoundry/mlfoundry_run.py", line 212, in _terminate_run_if_running
    current_status = self.status
  File "/Users/nikp18/opt/anaconda3/envs/p38/lib/python3.8/site-packages/mlfoundry/mlfoundry_run.py", line 149, in status
    return self.mlflow_client.get_run(self.run_id).info.status
  File "/Users/nikp18/opt/anaconda3/envs/p38/lib/python3.8/site-packages/mlflow/tracking/client.py", line 114, in get_run
    return self._tracking_client.get_run(run_id)
  File "/Users/nikp18/opt/anaconda3/envs/p38/lib/python3.8/site-packages/mlflow/tracking/_tracking_service/client.py", line 94, in get_run
    return self.store.get_run(run_id)
  File "

RUN ID: e6d5c80fab17497eb3366ac62f985fca


### Setting tags

In [46]:
clf = SVC(gamma='scale', kernel='rbf', probability=True, C=1.2)
run.set_tags({'framework': 'sklearn', 'task': 'classification'})

[mlfoundry] 2022-10-03T18:34:49+0530 INFO Tags set successfully


### Training model, logging parameters and logging model

In [47]:
clf.fit(X_train, y_train)
print(clf.get_params())
run.log_params(clf.get_params())
run.log_params({'classes': clf.classes_, 'features': clf.n_features_in_})
model_version=run.log_model(
    name="iris-model-demo",
    model=clf,
    framework=mlf.ModelFramework.SKLEARN,
    description="sklearn framework used"
)
print(model_version.fqn)

{'C': 1.2, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
[mlfoundry] 2022-10-03T18:34:49+0530 INFO Parameters logged successfully
[mlfoundry] 2022-10-03T18:34:50+0530 INFO Parameters logged successfully
[mlfoundry] 2022-10-03T18:34:50+0530 INFO Logging model and additional files, this might take a while ...
[mlfoundry] 2022-10-03T18:34:50+0530 INFO Serializing model files to model version contents
[mlfoundry] 2022-10-03T18:34:51+0530 WARNING pid=96743 thread=MLFSenderThread:6a6db3d611af403899aeb16b78c46b7d Encountered an internal error while trying to consume: UNAUTHORIZED: failed to get user info
[mlfoundry] 2022-10-03T18:34:52+0530 INFO Packaging and uploading files to remote ...
[mlfoundry] 2022-10-03T18:35:00+0530 INFO Logged model successfully with fqn 'model:tru

### Computing predictions

In [ ]:
# logging predictions
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

### Logging metrics

In [ ]:
metrics = {
    'train/accuracy_score': accuracy_score(y_train, y_pred_train),
    'train/f1_weighted': f1_score(y_train, y_pred_train, average='weighted'),
    'train/f1_mirco': f1_score(y_train, y_pred_train, average='micro'),
    'train/f1_macro': f1_score(y_train, y_pred_train, average='macro'),
    'test/accuracy_score': accuracy_score(y_test, y_pred_test),
    'test/f1_weighted': f1_score(y_test, y_pred_test, average='weighted'),
    'test/f1_mirco': f1_score(y_test, y_pred_test, average='micro'),
    'test/f1_macro': f1_score(y_test, y_pred_test, average='macro'),
}
print('Metrics', metrics)
run.log_metrics(metrics)

### Log the dataset

In [ ]:
run.log_dataset(
    dataset_name='train',
    features=X_train,
    predictions=y_pred_train,
    actuals=y_train,
)

In [ ]:
run.log_dataset(
    dataset_name='test',
    features=X_test,
    predictions=y_pred_test,
    actuals=y_test,
)

In [ ]:
run.end()